# Day 14: Extended Polymerization

## Part 1

### Define Functions

In [1]:
def parse(source):
    if source.endswith(".txt"):
        try:
            with open(source) as f:
                source = f.read().strip()
        except:
            print("file not found, please try again")
            return -1
    lines = source.split('\n')
    template = lines[0]
    rules = lines[2:]
    rule_dict = {}
    for rule in rules:
        rule_dict[rule[:2]] = rule[-1]
    return [template, rule_dict]

In [2]:
def polymer_growth(template, rules, num_steps = 1, verbose = False):
    if verbose:
        print("Template:", template)
    for step_num in range(num_steps):
        polymer = template[0]
        for i in range(len(template) - 1):
            pair = template[i:i+2]
            el_to_insert = rules[pair]
            polymer = polymer[:] + el_to_insert + template[i+1]
        template = polymer
        if verbose:
            print(f"After step {step_num+1}: length {len(polymer)}, {polymer}")
    return polymer

In [3]:
def most_minus_least_common(polymer, verbose = False):
    chars = list(set(sorted(polymer)))
    num_occur = {}
    for c in chars:
        num_occur[c] = polymer.count(c)
    maxq = max(num_occur.values())
    minq = min(num_occur.values())
    if verbose:
        print("element | quantity")
        for c, num in num_occur.items():
            print(f"{c} | {num}")
        print(f"most common ({(max(num_occur, key=num_occur.get))}, {maxq})")
        print(f"least common ({(min(num_occur, key=num_occur.get))}, {minq})")
    return maxq - minq

### Test on Example

In [4]:
example_input = """NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C"""

In [5]:
template_ex, rules_ex = parse(example_input)
template_ex

'NNCB'

In [6]:
rules_ex

{'CH': 'B',
 'HH': 'N',
 'CB': 'H',
 'NH': 'C',
 'HB': 'C',
 'HC': 'B',
 'HN': 'C',
 'NN': 'C',
 'BH': 'H',
 'NC': 'B',
 'NB': 'B',
 'BN': 'B',
 'BB': 'N',
 'BC': 'B',
 'CC': 'N',
 'CN': 'C'}

In [7]:
polymer_ex = polymer_growth(template_ex, rules_ex, 10)#, verbose = True)
most_minus_least_common(polymer_ex, verbose = True)

element | quantity
H | 161
C | 298
B | 1749
N | 865
most common (B, 1749)
least common (H, 161)


1588

### Input File

In [8]:
file_name = "InputFiles/day14input.txt"

In [9]:
template, rules = parse(file_name)

In [10]:
polymer = polymer_growth(template, rules, 10)
most_minus_least_common(polymer, verbose = True)

element | quantity
C | 1942
B | 671
F | 862
S | 2819
O | 2839
N | 2693
K | 1162
V | 2904
H | 2414
P | 1151
most common (V, 2904)
least common (B, 671)


2233

## Part 2

I knew this was too easy...

Lanternfish flashback... time to find a less expensive solution...

Instead of keeping track of the entire string, I'll only keep track of (1) the number of times each *pair* occurs in the string and (2) the number of times each *character* occurs in the string.

### New Function

In [11]:
#optimized polymer_growth function
def op_polymer_growth(template, rules, num_steps = 1, verbose = False):
    pair_count = {}
    element_count = {}
    
    #initalize pair_count and element_count to template
    for i, char in enumerate(template):
        if char in element_count:
            element_count[char] += 1
        else:
            element_count[char] = 1
        
        pair = template[i:i+2]
        if pair in pair_count:
            pair_count[pair] += 1
        elif len(pair) == 2:
            pair_count[pair] = 1
    
    # update pair_count and element_count with n steps
    for step in range(num_steps):
        
        # start with a copy of pair_count in its current form each step
        # this is what we will reference to determine quantity of each pair
        # this way, if a pair is created by the insertion process, it will - 
        # not count as the quantity at the beginning of the step
        
        this_step = pair_count.copy()
        
        for pair in this_step.keys():
            if this_step[pair] > 0:
                elem_to_add = rules[pair]
                n = this_step[pair]
                if elem_to_add in element_count:
                    element_count[elem_to_add] += n
                else:
                    element_count[elem_to_add] = n
                
                pair_count[pair] -= n
                
                if (pair[0] + elem_to_add) in pair_count:
                    pair_count[pair[0] + elem_to_add] += n
                else:
                    pair_count[pair[0] + elem_to_add] = n

                if (elem_to_add + pair[1]) in pair_count:
                    pair_count[elem_to_add + pair[1]] += n
                else:
                    pair_count[elem_to_add + pair[1]] = n
    
    return max(element_count.values()) - min(element_count.values())

### Example

In [12]:
op_polymer_growth(template_ex, rules_ex, 10)

1588

In [13]:
op_polymer_growth(template_ex, rules_ex, 40)

2188189693529

### Input File

In [14]:
op_polymer_growth(template, rules, 10)

2233

In [15]:
op_polymer_growth(template, rules, 40)

2884513602164